In [2]:
import copy
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import skimage
import glob
from tqdm import tqdm

In [11]:
class Verifier:
	def __init__(self, lowe_ratio=0.7, threshold=7):
		self.lowe_ratio = lowe_ratio
		self.threshold = threshold

	def __matcher(self, f1, f2):
		img1 =cv.imread(f1, cv.IMREAD_GRAYSCALE)
		img2 = cv.imread(f2, cv.IMREAD_GRAYSCALE)

		# Initiate SIFT detector
		sift = cv.SIFT_create()

		# find the keypoints and descriptors with SIFT
		kp1, des1 = sift.detectAndCompute(img1, None)
		kp2, des2 = sift.detectAndCompute(img2, None)
		FLANN_INDEX_KDTREE = 1
		index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
		search_params = dict(checks = 50)
		flann = cv.FlannBasedMatcher(index_params, search_params)
		matches = flann.knnMatch(des1,des2,k=2)
		# store all the good matches as per Lowe's ratio test.
		score = 0
		for m,n in matches:
			if m.distance < self.lowe_ratio*n.distance:
				score += 1

		return score

	def fit(self, x, y):
		self.x = np.array(x)
		self.y = np.array(y)

	def predict(self, x, y):
		idxs = np.where(self.y == y)
		score = sum(self.__matcher(ref, x) for ref in self.x[idxs])
		return score > self.threshold

In [49]:
def loadData(path):
	paths = np.array(glob.glob(path))
	ids = np.array([path.split('/')[1][2] + path.split('/')[-1].split("_")[0] for path in paths])

	train_data = []
	train_labels = []
	test_data = []
	test_labels = []

	# Split into train and test with 50-50 split
	for id in np.unique(ids):
		p = paths[ids == id]
		np.random.seed(2)
		np.random.shuffle(p)
		train_data.extend(p[4:])
		train_labels.extend(np.repeat(id, 4))
		test_data.extend(p[:4])
		test_labels.extend(np.repeat(id, 4))

	return train_data, train_labels, test_data, test_labels

In [57]:
train_data, train_labels, test_data, test_labels = loadData('data/DB1_B/*.tif')

v = Verifier(0.6, 20)
v.fit(train_data, train_labels)

confusion_mat = np.zeros((2, 2))

for i, (data, label) in tqdm(list(enumerate(zip(test_data, test_labels)))):
	parity = np.random.choice([0, 1])
	if parity == 0:
		label = np.random.choice(np.delete(np.unique(train_labels), np.where(np.unique(train_labels) == label)))
	pred = v.predict(data, label)
	confusion_mat[parity, int(pred)] += 1

print(confusion_mat)
print(f'Accuracy: {np.sum(np.diag(confusion_mat)) / np.sum(confusion_mat)}')

100%|██████████| 40/40 [00:18<00:00,  2.11it/s]

[[18.  0.]
 [ 4. 18.]]
Accuracy: 0.9


In [55]:
train_data, train_labels, test_data, test_labels = loadData('data/DB*_B/*.tif')

v = Verifier(0.6, 20)
v.fit(train_data, train_labels)

confusion_mat = np.zeros((2, 2))

for i, (data, label) in tqdm(list(enumerate(zip(test_data, test_labels)))):
	parity = np.random.choice([0, 1])
	if parity == 0:
		label = np.random.choice(np.delete(np.unique(train_labels), np.where(np.unique(train_labels) == label)))
	pred = v.predict(data, label)
	confusion_mat[parity, int(pred)] += 1

print(confusion_mat)
print(f'Accuracy: {np.sum(np.diag(confusion_mat)) / np.sum(confusion_mat)}')

100%|██████████| 160/160 [01:09<00:00,  2.32it/s]

[[78.  0.]
 [49. 33.]]
Accuracy: 0.69375
